In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import plotly.express as px

#### Preparacion de datos
* Columnas tomadas: 'Brand', 'Model', 'Precio', 'Km', 'Year', 'Interes_%'
* Calculo de Z-Scores

In [ ]:
df = pd.read_csv('../data/processed/csv/cleaned_final_csv_scrap_completo.csv', encoding='utf-8')

# Limpieza de planes.
term = 12

registros_unicos = df.loc[(df['Plazo'] == term),
    ['ID_Auto', 'Brand', 'Model', 
     'Precio', 'Km', 'Year', 'Interes_%',
     'Version', 'Caja', 'Tipo', 'Total_a_Pagar', 'Plazo']].copy()
registros_unicos = registros_unicos.sort_values(by=['Year', 'Km', 'Precio'], na_position='last')
registros_unicos = registros_unicos.dropna()
registros_unicos = registros_unicos.drop_duplicates(subset=['ID_Auto'], keep='first')
registros_unicos = registros_unicos.drop(columns=['ID_Auto'])

X = registros_unicos[['Precio', 'Km', 'Year', 'Interes_%']].copy()
#Promedios y DESV.EST
X_mean = X.mean()
X_std = X.std()

# Obtenemos el Z-Score para cada variable
X['Precio_z'] = (X['Precio'] - X_mean['Precio']) / X_std['Precio']
X['Km_z']     = (X['Km'] - X_mean['Km']) / X_std['Km']
X['Year_z']   = (X['Year'] - X_mean['Year']) / X_std['Year']
X['Interes_%_z'] = (X['Interes_%'] - X_mean['Interes_%']) / X_std['Interes_%']

#### Busqueda del numero de agrupamientos optimos. (Elbow method)

In [ ]:
X_scaled = X[['Precio_z', 'Km_z', 'Year_z']].copy()
inertia = []
K_range = range(1, 11)

for k in K_range:
    K_means = KMeans(n_clusters=k, random_state=97, n_init=10)
    K_means.fit(X_scaled)
    inertia.append(K_means.inertia_)

# Graficamos
plt.figure(figsize=(10,6))
plt.plot(K_range, inertia, marker='o', linestyle='--', color='blue')
plt.xlabel('Numero de Clusters (K)')
plt.ylabel('Inercia (Distancia intra-cluster)')
plt.title('Metodo del codo: Buscando el K optimo')
plt.xticks(K_range)
plt.grid(True)
plt.show()

#### Clustering con n = 3 

In [ ]:
k_means = KMeans(n_clusters=3, random_state=97, n_init=10)
k_means.fit(X_scaled)
pd.options.display.float_format = '{:,.0f}'.format

resultados_df = registros_unicos.copy()
resultados_df['Cluster'] = k_means.labels_

cluster_names = {
    0: 'Alto Kilometraje',
    1: 'Standard',
    2: 'Seminuevo'
}

resultados_df['Segmento'] = resultados_df['Cluster'].map(cluster_names)

profiles = resultados_df[['Precio', 'Km', 'Year', 'Interes_%', 'Segmento']].groupby('Segmento').mean().sort_values('Precio')
profiles['Cantidad_Autos'] = resultados_df['Segmento'].value_counts()

print("PERFILES DE LOS 3 GRUPOS DE AUTSOS EN KAVAK")
print(profiles)

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=resultados_df, x='Km', y='Precio', hue='Segmento', palette='viridis_r', alpha=0.6)
plt.axhline(y=350000)
plt.axvline(x=70000)
plt.title('Market Segmentation: 3 Tipos de Autos')
plt.grid(True, alpha=0.3)
plt.show()

#### Insights

In [ ]:
# TODO Integracion de una funcion para extraer datos con plazo especifico

In [ ]:
def explorar_modelo_interactivo(df, model):
    data = df[df['Model'] == model]

    fig = px.scatter(
        data,
        x='Interes_%',
        y='Km',
        color='Caja',
        symbol='Segmento',
        hover_data=['Precio', 'Year', 'Version', 'Total_a_Pagar', 'Plazo'],
        title=f'Analisis Profundo: {model}',
        template='plotly_white',
        height=700
    )

    fig.update_traces(marker=dict(size=10, line=dict(width=1, color='DarkSlateGrey')))
    fig.show()

explorar_modelo_interactivo(resultados_df, 'Gol')

In [ ]:
def cluster_brands_analysis(df, segmento, top_n=5):
    cluster_data = df[df['Segmento'] == segmento]

    top_brands = cluster_data['Brand'].value_counts().head(top_n)
    top_models = cluster_data['Model'].value_counts().head(top_n)
    
    print(f"--- Top Marcas en Cluster {segmento} ---")
    print(top_brands)
    print(f"\n--- Top Modelos en Cluster {segmento} ---")
    print(top_models)
    print("\n" + "="*40 + "\n")

In [ ]:
# Ejecutamos para los 3 clusters
cluster_brands_analysis(resultados_df, 'Alto Kilometraje') # Seminuevos (Morado)
cluster_brands_analysis(resultados_df, 'Standard') # Kilometraje Alto (Verde)
cluster_brands_analysis(resultados_df, 'Seminuevo') # Premium/Caros (Amarillo)

In [ ]:
sns.set_theme(style='ticks')
sns.pairplot(resultados_df, hue='Cluster')